## About Kosarak
- Distinct element: 606770
- Number of incomming element: 990002
- Top-10 frequent element and count: 

| element | frquent |
| :----- | :----- |
| 3 | 36133 |
| 6 | 30225 |
| 6 3 | 21139 |
| 11 6 | 12769 |
| 1 | 9766 |
| 11 6 3 | 9083 |
| 1 3 | 6002 |
| 11 | 5374 |
| 1 6 | 5046 |
| 4 | 4290 |


In [3]:
# count distinct element, set不能把list丟進去，要轉成tuple
import sys

num_line=0
count=0
item_set=set()
with open('kosarak.dat','r') as file:
    while True:
        line=tuple(file.readline().split())
        if not line:
            break
        else:
            num_line+=1
            if line not in item_set:
                item_set.add(line)
print("Distinct element: {}".format(len(item_set)))
print("Number of incomming element: {}".format(num_line))
print("Size of item_set: {}".format(sys.getsizeof(item_set)))

Distinct element: 606770
Number of incomming element: 990002
Size of item_set: 16777440


In [19]:
# count distinct element frequency
import pandas as pd 

num_line=0
count=0
itemdict={}
with open('kosarak.dat','r') as file:
    while True:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            num_line+=1         
            if line not in itemdict:
                itemdict[line]=1
            else:
                itemdict[line]+=1

df=pd.DataFrame(list(itemdict.items()),columns=['Element', 'Count'])
df=df.reset_index(drop=True)
df=df.sort_values('Count',ascending=False)

print("Distinct element: {}".format(len(itemdict)))
print("Number of incomming element: {}".format(num_line))
print(df.head(20))

df.to_csv("kosarak_distinct_count.csv",index=False)


Distinct element: 606770
Number of incomming element: 990002
       Element  Count
17           3  36133
111          6  30225
31         6 3  21139
94        11 6  12769
1            1   9766
85      11 6 3   9083
271        1 3   6002
253         11   5374
37         1 6   5046
279          4   4290
86        11 3   3756
241         55   3561
466      1 6 3   3510
233        4 3   3000
64        55 3   2377
195     11 1 6   2375
1525       4 6   2340
742      4 6 3   1758
250   11 1 6 3   1753
89        6 55   1749


'\nwith open("kosarak_distinct_count.txt",\'w\') as write_obj:\n    for item in sorted(itemdict.items(),key = lambda s: s[1],reverse=True):\n        temp=list(item)\n        tempstr=temp[0]+\',\'+str(temp[1])+\'\n\'\n        write_obj.write(tempstr)\n'

## Count-min sketch
- Top-256 with size 2424 bytes.
- Execution time:1.4817874431610107 seconds.
- Top-512 with size 4728 bytes.
- Execution time:228.55175113677979 seconds.

In [18]:
# count min sketch with kosarak, keeps top-256 elements,read 10000 items.

import sys
import time
from probables import (CountMinSketch)

start=time.time()
topk=[]
size=256
item=10000
cms = CountMinSketch(width=1024, depth=4)
with open("kosarak.dat",'r') as file:
    while item:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            item-=1
            cms.add(line)
            count=cms.check(line)
            if len(topk)==0:
                topk.append([line,count])
            else:
                find=False
                for i in range(len(topk)):
                    if line==topk[i][0]:
                        topk[i][1]=count
                        topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                        find=True
                        break
                if find==False:
                    if len(topk)<size:
                        topk.append([line,count])
                        topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                    else:
                        topk.append([line,count])
                        topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                        topk.pop()
end=time.time()
print(topk[:64],len(topk))
print("Top-{} with size {} bytes.".format(len(topk),sys.getsizeof(topk)))
print("Execution time:{} seconds.".format(str(end-start)))

[['3', 383], ['6', 292], ['6 3', 214], ['11 6', 158], ['1', 122], ['11 6 3', 95], ['1 3', 64], ['11 3', 58], ['1 6', 56], ['4', 49], ['11', 47], ['55', 42], ['1 6 3', 39], ['4 3', 33], ['55 3', 31], ['4 6 3', 25], ['11 1 6', 24], ['2', 21], ['4 6', 21], ['294', 21], ['11 4 6', 20], ['294 3', 20], ['6 55 3', 20], ['7', 19], ['6 55', 17], ['11 6 2', 17], ['218', 17], ['6 2', 16], ['11 1 6 3', 16], ['11 6 2 3', 16], ['2 3', 16], ['648', 16], ['6 294 3', 16], ['490', 15], ['27', 15], ['11 1', 15], ['73', 15], ['11 4 6 3', 14], ['6 2 3', 14], ['11 6 55 3', 14], ['490 3', 14], ['64', 13], ['77', 13], ['11 1 3', 13], ['1 55 9085 444', 13], ['3 316', 13], ['6 7', 12], ['218 6 1657 1658 3665 303 650 827 3436 91 9907 3 7 77 531 737 467', 12], ['490 6', 12], ['316', 12], ['6 3 77', 12], ['4 1', 11], ['138', 11], ['3693', 11], ['11 27 1 6 1898 860 1900 1769 3492 1914 3794 2342 215 1800 1801 2952 826 1131 5013 3 1714 1956 1496 5344 1499 205 5990 2805 1957 9761 9762 9763 3643 5929 9764 9765 9766 195

In [1]:
# count min sketch with kosarak, keeps top-1024, full read

import sys
import time
from probables import (CountMinSketch)

start=time.time()
topk=[]
size=1024
cms = CountMinSketch(width=1024, depth=4)
with open("kosarak.dat",'r') as file:
    while True:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            cms.add(line)
            count=cms.check(line)
            if len(topk)==0:
                topk.append([line,count])
            else:
                find=False
                for i in range(len(topk)):
                    if line==topk[i][0]:
                        topk[i][1]=count
                        topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                        find=True
                        break
                if find==False:
                    if len(topk)<size:
                        topk.append([line,count])
                        topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                    else:
                        topk.append([line,count])
                        topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                        topk.pop()
end=time.time()
print(topk[:100],len(topk))
print("Top-{} with size {} bytes.".format(len(topk),sys.getsizeof(topk)))
print("Execution time:{} seconds.".format(str(end-start)))

[['3', 36826], ['6', 30906], ['6 3', 21876], ['11 6', 13465], ['1', 10512], ['11 6 3', 9726], ['1 3', 6663], ['11', 6049], ['1 6', 5695], ['4', 4969], ['11 3', 4564], ['55', 4269], ['1 6 3', 4156], ['4 3', 3666], ['55 3', 3050], ['11 1 6', 3005], ['4 6', 2992], ['4 6 3', 2441], ['11 1 6 3', 2437], ['6 55', 2430], ['2', 2310], ['294', 2276], ['6 55 3', 2004], ['490', 1981], ['2 3', 1946], ['27', 1822], ['7', 1770], ['294 3', 1759], ['218', 1717], ['11 4 6', 1594], ['490 3', 1578], ['11 1', 1554], ['6 2', 1540], ['6 294', 1534], ['64', 1521], ['73', 1495], ['11 4 6 3', 1488], ['91', 1488], ['4 1', 1438], ['218 3', 1434], ['11 6 55', 1412], ['11 6 55 3', 1409], ['6 2 3', 1389], ['3 7', 1373], ['490 6', 1365], ['6 7', 1365], ['6 294 3', 1354], ['27 3', 1349], ['218 6', 1324], ['77', 1311], ['138', 1309], ['316', 1307], ['11 1 3', 1300], ['1 55', 1298], ['27 6', 1291], ['3 73', 1280], ['3 64', 1203], ['27 6 3', 1201], ['6 3 7', 1196], ['3 316', 1175], ['4 1 3', 1171], ['1 218 55 1065 2866 1

In [3]:
import sys
print(sys.getsizeof(topk)+sys.getsizeof(cms))

9432


## Space-Saving
- Top-1024 with size 9328 bytes.
- Execution time:3.5773608684539795 seconds.
- Top-512 with size 4720 bytes.
- Execution time:150.65503454208374 seconds.

In [3]:
# space saving kosarak, itemlist[i][0]為讀入的行，不為list,
# keeps top-256 elements,read 10000 items.

import sys
import time
start=time.time()

size=1024
itemlist=[]
item=100
with open("kosarak.dat",'r') as file:
    while True:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            item-=1
            if len(itemlist)==0:
                itemlist.append([line,1])
            else:
                for i in range(len(itemlist)):
                    if line==itemlist[i][0]:
                        itemlist[i][1]+=1
                        found=True
                        #print("found in itemlist[{}]".format(i))
                        break
                    else:
                        found=False
                if found:
                    pass
                else:
                    if len(itemlist)<size:
                        itemlist.append([line,1])
                    else:
                        # len(itemset)=size
                        last=len(itemlist)-1
                        itemlist[last][0]=line
                        itemlist[last][1]+=1
            itemlist=sorted(itemlist,key = lambda itemlist: itemlist[1],reverse=True)
        #print("itemlist:{}\nlen(itemlist):{}\n".format(itemlist,len(itemlist)))
end=time.time()

print(itemlist[:100],len(itemlist))
print("Top-{} with size {} bytes.".format(len(itemlist),sys.getsizeof(itemlist)))
print("Execution time:{} seconds.".format(str(end-start)))

KeyboardInterrupt: 

In [17]:
# space saving kosarak, 讀入的line會變成list
import sys
size=200
itemlist=[]
income=0
with open("kosarak.dat",'r') as file:
    while True:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            income+=1
            temp=line.split()
            if len(itemlist)==0:
                itemlist.append([temp,1])
            else:
                for i in range(len(itemlist)):
                    if temp==itemlist[i][0]:
                        itemlist[i][1]+=1
                        found=True
                        #print("found in itemlist[{}]".format(i))
                        break
                    else:
                        found=False
                if found:
                    pass
                else:
                    if len(itemlist)<size:
                        itemlist.append([temp,1])
                    else:
                        # len(itemset)=size
                        last=len(itemlist)-1
                        temp=itemlist[last][0]
                        itemlist[last][0]=line
                        itemlist[last][1]+=1
            itemlist=sorted(itemlist,key = lambda itemlist: itemlist[1],reverse=True)
        #print("itemlist:{}\nlen(itemlist):{}\n".format(itemlist,len(itemlist)))
print(itemlist[:10])
print(sys.getsizeof(itemlist),'bytes')

[[['3'], 36133], [['6'], 30225], [['6', '3'], 21139], [['11', '6'], 12769], [['1'], 9766], [['11', '6', '3'], 9083], ['766 3', 4490], ['11 6 55 514 278 28 7 1834 400 32 438', 4490], ['11 6 3 3262', 4490], ['218', 4490]]
1912 bytes


In [4]:
# space saving kosarak, top-1024 elements, full test
# Execution time:303.9870328903198 seconds

import sys
import time
start=time.time()

size=1024
itemlist=[]
with open("kosarak.dat",'r') as file:
    while True:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            if len(itemlist)==0:
                itemlist.append([line,1])
            else:
                for i in range(len(itemlist)):
                    if line==itemlist[i][0]:
                        itemlist[i][1]+=1
                        found=True
                        #print("found in itemlist[{}]".format(i))
                        break
                    else:
                        found=False
                if found:
                    pass
                else:
                    if len(itemlist)<size:
                        itemlist.append([line,1])
                    else:
                        # len(itemset)=size
                        last=len(itemlist)-1
                        itemlist[last][0]=line
                        itemlist[last][1]+=1
            itemlist=sorted(itemlist,key = lambda itemlist: itemlist[1],reverse=True)
        #print("itemlist:{}\nlen(itemlist):{}\n".format(itemlist,len(itemlist)))
end=time.time()

print(itemlist[:100],len(itemlist))
print("Top-{} with size {} bytes.".format(len(itemlist),sys.getsizeof(itemlist)))
print("Execution time:{} seconds.".format(str(end-start)))

[['3', 36133], ['6', 30225], ['6 3', 21139], ['11 6', 12769], ['1', 9766], ['11 6 3', 9083], ['1 3', 6002], ['11', 5374], ['1 6', 5046], ['4', 4290], ['11 3', 3756], ['55', 3561], ['1 6 3', 3510], ['4 3', 3000], ['55 3', 2377], ['11 1 6', 2375], ['4 6', 2341], ['4 6 3', 1758], ['11 1 6 3', 1753], ['6 55', 1749], ['294', 1653], ['2', 1601], ['6 55 3', 1375], ['490', 1226], ['27', 1168], ['294 3', 1162], ['2 3', 1152], ['7', 1100], ['218', 1056], ['11 4 6', 954], ['64', 915], ['6 294', 893], ['6 2', 882], ['11 1', 880], ['91', 866], ['73', 853], ['490 3', 846], ['11 4 6 3', 828], ['6 294 3', 823], ['4 1', 821], ['6 2 3', 821], ['3 73', 820], ['27 6 3', 820], ['27 3', 819], ['6 7', 819], ['11 6 55', 819], ['4 1 3', 818], ['218 3', 818], ['3 7', 818], ['316', 818], ['18', 818], ['11 6 55 3', 817], ['11 218 6 3', 817], ['4 55 3', 817], ['278', 817], ['584', 817], ['6 91', 817], ['142 3', 817], ['6 504 3', 817], ['1056', 817], ['4 1 6', 816], ['1 55', 816], ['11 1 3', 816], ['667', 816], ['1